# NeuralTreeNet and ONNX

La conversion d'un arbre de décision au format ONNX peut créer des différences entre le modèle original et le modèle converti (voir [Issues when switching to float](http://www.xavierdupre.fr/app/onnxcustom/helpsphinx/gyexamples/plot_ebegin_float_double.html). Le problème vient d'un changement de type, les seuils de décisions sont arrondis au float32 le plus proche de leur valeur en float64 (double). Qu'advient-il si l'arbre de décision est converti en réseau de neurones d'abord.

L'approximation des seuils de décision ne change pas grand chose dans la majorité des cas. Cependant, il est possible que la comparaison d'une variable à un seuil de décision arrondi soit l'opposé de celle avec le seuil non arrondi. Dans ce cas, la décision suit un chemin différent dans l'arbre.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

## Jeu de données

On construit un jeu de donnée aléatoire.

In [3]:
import numpy

X = numpy.random.randn(10000, 10)
y = X.sum(axis=1) / X.shape[1] + numpy.random.randn(X.shape[0]) / 2000
X = X.astype(numpy.float64)
y = y.astype(numpy.float64)

In [4]:
middle = X.shape[0] // 2
X_train, X_test = X[:middle], X[middle:]
y_train, y_test = y[:middle], y[middle:]

## Caler un arbre de décision

In [5]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=10)
tree.fit(X_train, y_train)
tree.score(X_train, y_train), tree.score(X_test, y_test)

(0.8703013749822193, 0.416418893707294)

In [6]:
from sklearn.metrics import r2_score
r2_score(y_test, tree.predict(X_test))

0.416418893707294

## Conversion au format ONNX

In [7]:
from mlprodict.onnx_conv import to_onnx

onx = to_onnx(tree, X[:1].astype(numpy.float32))

In [8]:
from mlprodict.onnxrt import OnnxInference

x_exp = X_test

oinf = OnnxInference(onx, runtime='onnxruntime1')
expected = tree.predict(x_exp)
got = oinf.run({'X': x_exp.astype(numpy.float32)})['variable']
numpy.abs(got - expected).max()

2.035147748808148